# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [23]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [24]:
# This should make a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [25]:
# Create a Keyspace 
KEYSPACE = "sparkify"
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS %s
    WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
    """ % KEYSPACE)

#### Set Keyspace

In [26]:
#Set KEYSPACE to the keyspace specified above
session.set_keyspace(KEYSPACE)

# Table 1

Because we will retrieve sessionId, and itemInSession. We will need to make `sessionId` and `itemInSession` part of the primary key. `sessionId` is a primary key, and `itemInSession` is a clustering key.

In [37]:
create_table1 = """
    CREATE TABLE session_music (
        sessionId int, 
        itemInSession int, 
        artist text, 
        song text, 
        length float, 
        PRIMARY KEY (sessionId, itemInSession ) )"""
try: 
    session.execute(create_table1)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

Table Created!!


In [38]:
# We have provided part of the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_music(sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

# Query 1

Give me the artist, song title and song's length in the music app history that was heard during 
`sessionId = 338`, and `itemInSession = 4`

In [39]:
 
query1 = """SELECT 
    artist, 
    song, 
    length FROM session_music WHERE sessionId=338 AND itemInSession=4"""
try: 
    rows = session.execute(query1)
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


# Table 2 

Create table to support querying songs by session.

`userId` and `sessionId` are partition keys and `itemInSession` is clustering key.

In [40]:
# Create table
create_table2 = """
        CREATE TABLE IF NOT EXISTS song_per_session (
            userId INT,
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            firstName TEXT,
            lastName TEXT,
            PRIMARY KEY ((userId, sessionId), itemInSession));
    """
try: 
    session.execute(create_table2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")


                    

Table Created!!


In [41]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_per_session (userId, sessionId, itemInSession, artist, song, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

 # Query 2
Give me only the following: name of `artist`, `song` (sorted by `itemInSession`) and user (`firstName` and `lastName`) for `userId = 10`, `sessionId = 182`


In [42]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = """SELECT artist, song , firstName, lastName 
            FROM song_per_session 
            WHERE userId=10 AND sessionId=182
            ORDER BY itemInSession"""
try: 
    rows = session.execute(query2)
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


# Table 3

Create table to support querying song listening history per user by song name.

`song` is a partition key. `userId` is clustering keys.


In [45]:
create_table3 = """
        CREATE TABLE IF NOT EXISTS song_listen_history (
            firstName TEXT,
            lastName TEXT,
            userId INT,
            song TEXT,
            PRIMARY KEY (song, userId) );
    """
try: 
    session.execute(create_table3)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")


Table Created!!


In [46]:
# We have provided part of the code to set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listen_history(song, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# Query 3

Give me user `firstName` and `lastName` in my music app song listen history who listened to the `song` 'All Hands Against His Own'


In [48]:
query3 = """SELECT firstName, lastName FROM song_listen_history WHERE song='All Hands Against His Own' """
try: 
    rows = session.execute(query3)
except Exception as e:
    print(f"Table creation failed!! Error : {e}")

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [49]:
try:
    session.execute("DROP TABLE session_music")
    session.execute("DROP TABLE song_per_session")
    session.execute("DROP TABLE song_listen_history")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()